<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/BTC_BITMEX_SMAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bitmex
!pip install backtesting

import bitmex
import requests
import pandas as pd
from backtesting import Backtest, Strategy

API_KEY = "AxrCpuvGBTUCvzCPW-UcB2Le"
API_SECRET = "HkS2XQb86BV4HQzVnfIV3Ez4dNim8AjIIQYJU3zIf455umIH"

# Create authenticated BitMEX client
client = bitmex.bitmex(api_key=API_KEY, api_secret=API_SECRET)

def get_bitmex_data(symbol, timeframe):
    base_url = "https://www.bitmex.com/api/v1/trade/bucketed"
    params = {
        "binSize": timeframe,
        "symbol": symbol,
        "count": 1000,
        "reverse": "true"
    }
    response = requests.get(base_url, params=params)
    data = response.json()  # Extract the JSON data
    df = pd.DataFrame(data)

    # Convert 'timestamp' to datetime and set as index
    print(df.head())

    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.set_index('timestamp', inplace=True)

    # Rename columns
    df = df.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"})
    return df

class BitMEXStrategy(Strategy):
    def init(self):
        close = self.data.Close
        self.sma1 = self.I(close.rolling, window=20, min_periods=1).mean()
        self.sma2 = self.I(close.rolling, window=60, min_periods=1).mean()

    def next(self):
        if self.sma1[-1] > self.sma2[-1] and self.sma1[-2] < self.sma2[-2]:
            # Buy signal
            if self.position.is_short():
                self.position.close()  # Close short position
            self.buy()

            # Place a buy order on BitMEX
            client.Order.Order_new(symbol='XBTUSD', orderQty=1, side="Buy").result()

        elif self.sma1[-1] < self.sma2[-1] and self.sma1[-2] > self.sma2[-2]:
            # Sell signal
            if self.position.is_long():
                self.position.close()  # Close long position
            self.sell()

            # Place a sell order on BitMEX
            client.Order.Order_new(symbol='XBTUSD', orderQty=1, side="Sell").result()

df = get_bitmex_data('XBTUSD', '4h')
bt = Backtest(df, BitMEXStrategy)
bt.run()
bt.plot()
